Das Programm soll eine Liste mit allen Motionen, Interpellationen und Postulaten erstellen, die im aktuellen Jahr (oder später in einer einstellbaren Zeitperiode) im Grossen Rat Aargau eingereicht wurden.
Es soll ausgewertet werden, welche Parlamentarier wieviele Vorstösse eingereicht haben und welche Parlamentarier und Parteien besonders oft zusammengearbeitet haben

In [63]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
import time
import os
import re
import pandas as pd
from tqdm import tqdm
import urllib.parse
from pathlib import Path
import numpy as np

In [64]:
#mit firstyeahr und lastyeahr soll später die Zeitspanne angegeben werden, in der die Geschäfte ausgewertete werden
firstyear = 2021
lastyear = 2021

#FUNKTIONEN UND LISTE MIT BEZIRKEN-GEMEINDEN

#Funktion, um alle weiteren Seiten abzuspeichern
def save_site(page,count):
    driver.find_element_by_xpath(page).click()
    time.sleep(2)
    page = driver.page_source.encode('utf-8')
    file = open("pages/parl/parl"+str(count)+".htm", "wb+")
    file.write(page)
    file.close()
    count += 1
    return count


#Funktion, um alle weiteren Seiten abzuspeichern
def save_site_vorstoesse(page_vorstoesse,count_vorstoesse):
    driver.find_element_by_xpath(page_vorstoesse).click()
    time.sleep(2)
    page = driver.page_source.encode('utf-8')
    file = open("pages/vorstoesse/"+adrid+"/"+str(count_vorstoesse)+".htm", "wb+")
    file.write(page)
    file.close()
    count_vorstoesse += 1
    return count_vorstoesse

#Hier bilde ich aus der Geschäftsnummer-ID das Jahr des Geschäfts
def trennen(elem):
    datum = elem.split(".")[0]
    datum = datum.replace('="', "")
    datum = datum_umwandeln(datum)
    return datum

#Mit dieser Funktion wird die zweistellige Zahl aus der Geschäfts-ID in eine 4-stellige Jahreszahl umgewandelt
def datum_umwandeln(jahr):
    jahr_neu = pd.to_datetime(jahr, format='%y')
    jahr_neu = jahr_neu.strftime("%Y")
    return jahr_neu

#Mit der Funktion prüfen wir, ob eine (oder mehrere) Fraktion(en) an einem Vorstoss beteiligt sind
def check_fraktionen(id_vorstoss,fraktionen):
    a = list_vorstoesse_namen[list_vorstoesse_namen["ID Vorstoss"] == id_vorstoss]["Fraktion"]
    #Nur eine Partei
    if len(fraktionen) == 1:
        if fraktionen[0] in a.unique():
            result=1
        else:
            result=0
    #2 Parteien
    if len(fraktionen) == 2:
        if fraktionen[0] in a.unique() and fraktionen[1] in a.unique():
            result=1
        else:
            result=0
    #3 Parteien
    if len(fraktionen) == 3:
        if fraktionen[0] in a.unique() and fraktionen[1] in a.unique() and fraktionen[2] in a.unique():
            result=1
        else:
            result=0
    return(result)

#(AG) aus den Ortsnahmen entfernen
def ag_entf(gemeinde):
    gemeinde = gemeinde.replace(' (AG)', "")
    return gemeinde

#Dict mit allen Bezirken, mit Listen aller Gemeinden
bezirke = {"Aarau":[],"Baden":[],"Bremgarten":[],"Brugg":[],"Kulm":[],"Laufenburg":[],"Lenzburg":[],"Muri":[],"Rheinfelden":[],"Zofingen":[],"Zurzach":[]}
for bezirk in bezirke:
    if bezirk == "Baden":
        adresse = "https://de.wikipedia.org/wiki/Bezirk_"+bezirk+"_(Aargau)"
    else:
        adresse = "https://de.wikipedia.org/wiki/Bezirk_"+bezirk
    res=pd.read_html(adresse)
    bezirke[bezirk] = res[2]["Name der Gemeinde"].apply(ag_entf)
    bezirke[bezirk].pop(len(bezirke[bezirk])-1)

#Funktion, um Bezirk zuzuordnen
def def_bezirk(gemeinde):
    for bezirk in bezirke:
        if gemeinde in bezirke[bezirk].unique():
            return "Bezirk "+bezirk
        
#Funktion, um Vorstösse zu zählen
def anzahl_vorstoesse(name):
    anzahl = list_vorstoesse_namen[list_vorstoesse_namen["Name"]==name].count()[0]
    return anzahl

#Funktion, um anhand der Partei die Fraktion zu bestimmen
def fraktion(partei):
    if partei == "SVP" or partei == "EDU":
        fraktion = "SVP/EDU"
    else:
        fraktion = partei
    return fraktion

In [65]:
#Zuerst werden alle Parlamentarier erfasst. 
#Dazu werden die entsprechenden Websiten heruntergeladen und jeder Parlamentarier als Dict  mit Partei/Ort/Link/ID erstellt 

#Neues Fenster mit 1. Seite aufrufen
driver = webdriver.Firefox()
driver.get('https://www.ag.ch/grossrat/grweb/de/164/Ratsmitglieder?')
#Variable count definieren, für Funktion save_site
count = 2

    
#Seite 1 abspeichern
page = driver.page_source.encode('utf-8')
file = open("pages/parl/parl1.htm", "wb+")
file.write(page)
file.close()


#Seiten 2-6
sites = ["/html/body/div/div/div/div/div/article/nav[1]/p[2]/a[1]",
        "/html/body/div/div/div/div/div/article/nav[1]/p[2]/a[4]",
        "/html/body/div/div/div/div/div/article/nav[1]/p[2]/a[5]",
        "/html/body/div/div/div/div/div/article/nav[1]/p[2]/a[5]",
        "/html/body/div/div/div/div/div/article/nav[1]/p[2]/a[6]"]

#Für alle Seiten Funktion save_site aufrufen, und count hochzählen
for site in sites:
    #print(site)
    count = save_site(site,count)

#falls Probleme auftauchen, driver.quit ist noch nicht getestet!!
driver.quit()

In [66]:
#Im Ordner auslesen, welche Seiten gespeichert wurden
lst = os.listdir('pages/parl/')

#Leere Liste erstellen. Hier kommen die Inhalte der Seiten rein
list_of_pages=[]

#Alle gespeicherten Seiten in die Liste list_of_pages schreiben
for i in range(0,len(lst)):
    file = open('pages/parl/' + lst[i], 'r', encoding='utf-8')
    text = file.read()
    list_of_pages.append(BeautifulSoup(text, 'html.parser'))

In [67]:
#Leere Liste erstellen. Hier kommen alle Politiker als Minidict rein (Name,Partei, Ort, Link)
politicians = []

#Für jede Seite, die in list_of_pages abgespeichert ist
for page in list_of_pages:
    result = page.find_all('table', {'class', 'table'})[0]

    #Suche in der jeder Zeile der Tabelle nach den entsprechenden Einträgen, speichere als Minidict in die Liste
    for politician in result.find_all('tr')[1:]:
        link = politician.find_all('a')[0].get('href')
        #Ich suche nach 'AdrId=', gefolgt von Zahlen. Durch die Klammern wird der String aufgeteilt, mit group(1) gebe ich nur die Zahlen zurück
        id_pol = re.search(r"AdrId=(\d+)", link).group(1)
        name = politician.find_all('td')[1].text
        #Es ist ein Problem aufgetaucht: Auf der von mir gespeicherten Seite wird nicht die Partei, sondern die Fraktion mitgegeben.
        #Das hat zur Folge, dass die beiden Politiker der EDU als Parteibezeichnung SVP bekommen haben. Ich korrigiere das vorläufig "von Hand":
        if name == "Bossert Martin" or name == "Haller Rolf":
            par = "EDU"
        else:
            par = politician.find_all('td')[0].text
        place = politician.find_all('td')[4].text
        #print(link)
        mini_dict = {"Name":name, "Partei":par, "Ort":place, "Link":link, "ID Politiker":id_pol}
        politicians.append(mini_dict)
        try:
            os.makedirs("pages/"+id_pol)
        except FileExistsError:
            # directory already exists
            pass


In [79]:
pd.set_option('display.max_colwidth', 100)
list_politicians = pd.DataFrame(politicians)

In [77]:
anzahl_gr = 0
while anzahl_gr < len(politicians):  
    adrid = list_politicians["ID Politiker"][anzahl_gr]
    
    #Neues Fenster mit 1. Seite aufrufen
    driver = webdriver.Firefox()
    driver.get('https://www.ag.ch/grossrat/grweb/de/185/Detail%20Adresse?AdrId='+adrid)
    #Variable count definieren, für Funktion save_site
    count = 2
    #2 Sekunden warten
    time.sleep(2)

    #Seite 1 abspeichern
    page = driver.page_source.encode('utf-8')
    if os.path.exists("pages/vorstoesse/"+adrid):
               file = open("pages/vorstoesse/"+adrid+"/1.htm", "wb+")
               file.write(page)
               file.close()
    else:
        os.makedirs("pages/vorstoesse/"+adrid)
        file = open("pages/vorstoesse/"+adrid+"/1.htm", "wb+")
        file.write(page)
        file.close()

    #file wieder öffnen, "Vorstösse" suchen und Zahl in Klammer speichern
    file = open("pages/vorstoesse/"+adrid+"/1.htm","r",encoding="UTF-8")
    text = file.read()
    h = BeautifulSoup(text, 'html.parser')
    text = h.find_all('caption')[1]
    anzahl = re.findall(r"\(([A-Za-z0-9_]+)\)", str(text)) 
    anzahl = int(anzahl[0])
    file.close()

    sites_vorstoesse = []

    if (anzahl > 25) and (anzahl <=50):
        sites_vorstoesse = ["/html/body/div/div/div/div/div/article/nav[1]/p[2]/a[1]"]

    if (anzahl > 50) and (anzahl <=75):
        sites_vorstoesse = ["/html/body/div/div/div/div/div/article/nav[1]/p[2]/a[1]",
                "/html/body/div/div/div/div/div/article/nav[1]/p[2]/a[4]"]

    if anzahl > 75 and anzahl <=100:
        sites_vorstoesse = ["/html/body/div/div/div/div/div/article/nav[1]/p[2]/a[1]",
                "/html/body/div/div/div/div/div/article/nav[1]/p[2]/a[4]",
                "/html/body/div/div/div/div/div/article/nav[1]/p[2]/a[5]",]

    if anzahl > 100 and anzahl <=125:
        sites_vorstoesse = ["/html/body/div/div/div/div/div/article/nav[1]/p[2]/a[1]",
                "/html/body/div/div/div/div/div/article/nav[1]/p[2]/a[4]",
                "/html/body/div/div/div/div/div/article/nav[1]/p[2]/a[5]",
                "/html/body/div/div/div/div/div/article/nav[1]/p[2]/a[5]"]

    if anzahl > 125 and anzahl <=150:
        sites_vorstoesse = ["/html/body/div/div/div/div/div/article/nav[1]/p[2]/a[1]",
                "/html/body/div/div/div/div/div/article/nav[1]/p[2]/a[4]",
                "/html/body/div/div/div/div/div/article/nav[1]/p[2]/a[5]",
                "/html/body/div/div/div/div/div/article/nav[1]/p[2]/a[5]",
                "/html/body/div/div/div/div/div/article/nav[1]/p[2]/a[6]"]

    if anzahl > 150:
        print("ACHTUNG!!! Anzahl grösser 150")

    #Für alle Seiten Funktion save_site aufrufen, und count hochzählen
    count_vorstoesse = 2
    for site_vorstoesse in sites_vorstoesse:
        count_vorstoesse = save_site_vorstoesse(site_vorstoesse,count_vorstoesse)
        
    anzahl_gr+=1
print("Ich bin fertig")
driver.quit()

Ich bin fertig


In [80]:
#Bezirke zuordnen
list_politicians['Ort'] = list_politicians['Ort'].apply(ag_entf)
list_politicians['Bezirk'] = list_politicians['Ort'].apply(def_bezirk)

#Fraktion reinschreiben
list_politicians["Fraktion"] = list_politicians["Partei"].apply(fraktion)

In [99]:
#Überprüfen
list_politicians[list_politicians["Bezirk"].isnull() == True]

,Name,Partei,Ort,Link,ID Politiker,Bezirk,Fraktion,Anzahl Vorstösse


In [85]:
#Korrigieren
orte_korr = {
    'Baden-Dättwil': 'Baden',
    'Kirchdorf': 'Obersiggenthal',
    'Hettenschwil': 'Leuggern',
    'Wildegg': 'Möriken-Wildegg',
    'Glashütten': 'Murgenthal',
    'Kleindöttingen': 'Böttstein',
    'Gippingen': 'Leuggern',
    'Nussbaumen b.Baden': 'Obersiggenthal',
    'Umiken': 'Brugg',
    'Baden 1': 'Baden',
    'Mühlethal': 'Zofingen',
}
list_politicians['Ort'].replace(orte_korr, inplace=True)


#erneut zuordnen
list_politicians['Bezirk'] = list_politicians['Ort'].apply(def_bezirk)

#Nicht herausgefunden, warum bei Leila Hunziker, Aarau, (id 63) der Bezirk nicht zugeordnet wird, deshalb von Hand:
list_politicians.at[63,'Bezirk']='Bezirk Aarau'

In [100]:
#Politiker in ein csv schreiben (für Auswertung Redezeit)
list_politicians.to_csv('aktive_parlamentarier.csv', index=False)

In [101]:
anzahl_gr = 0
#Leere Liste erstellen. Hier kommen alle Vorstösse als Minidict rein
vorstoesse = []
while anzahl_gr < len(politicians):  
    adrid = list_politicians["ID Politiker"][anzahl_gr]    
    #Im Ordner auslesen, welche Seiten gespeichert wurden
    lst = os.listdir('pages/vorstoesse/'+adrid)
    #print(lst)

    #Leere Liste erstellen. Hier kommen die Inhalte der Seiten rein
    list_of_pages_vorstoesse=[]

    #Alle gespeicherten Seiten in die Liste list_of_pages schreiben
    for i in range(0,len(lst)):
        file = open('pages/vorstoesse/'+adrid+"/" + lst[i], 'r', encoding='utf-8')
        text = file.read()
        list_of_pages_vorstoesse.append(BeautifulSoup(text, 'html.parser'))



    #Für jede Seite, die in list_of_pages abgespeichert ist
    for page in list_of_pages_vorstoesse:
        result = page.find_all('table', {'class', 'table'})[2]

        #Suche in der jeder Zeile der Tabelle nach den entsprechenden Einträgen, speichere als Minidict in die Liste
        for vorstoss in result.find_all('tr')[1:]:
            id_vorstoss = vorstoss.find_all('td')[0].text
            #vorstoesse.append(id_vorstoss)
            text_vorstoss = vorstoss.find_all('td')[1].text
            art_vorstoss = vorstoss.find_all('td')[2].text
            mini_dict = {"ID Vorstoss":id_vorstoss, "ID Politiker":adrid, "Art":art_vorstoss, "Text":text_vorstoss}
            vorstoesse.append(mini_dict)
            
    anzahl_gr+=1
print("Ich bin feeeidsch")

list_vorstoesse = pd.DataFrame(vorstoesse)

Ich bin feeeidsch


In [102]:
#Zuerst die Jahreszahl rauslsen und in eine neue Spalte schreiben
list_vorstoesse['Jahr'] = list_vorstoesse['ID Vorstoss'].apply(trennen).astype(int)

#Nur den Teil aus der Zeitspanne rausfiltern, der oben angegeben ist
list_vorstoesse = list_vorstoesse[(list_vorstoesse['Jahr'] >= firstyear) & (list_vorstoesse['Jahr'] <= lastyear)]

In [103]:
#Die Liste der Vorstösse mit der Liste der Politiker kombinieren, Link rauslöschen
list_vorstoesse_namen = list_vorstoesse.merge(list_politicians, how='inner', on='ID Politiker')
list_vorstoesse_namen  = list_vorstoesse_namen.drop(["Link"], axis=1)

In [104]:
### AUSWERTUNG

#Hier kann man mehrere Parteien angeben, es wird die Anzahl gemeinsamer Vorstösse zurückgegeben
anzahl_gemeinsam = 0
fraktionen=["Die Mitte","SVP/EDU"]
for ID_Vorstoss in list_vorstoesse_namen["ID Vorstoss"].unique():
    anzahl_gemeinsam+=check_fraktionen(ID_Vorstoss,fraktionen)
    
anzahl_gemeinsam
    

19

In [105]:
#In die Liste der Politiker die Anzahl Vorstösse reinschreiben
list_politicians['Anzahl Vorstösse'] = list_politicians['Name'].apply(anzahl_vorstoesse)

In [107]:
#File für jeden Bezirk erstellen
for bezirk in bezirke:
    file = list_politicians[list_politicians["Bezirk"]=="Bezirk "+bezirk].sort_values(by=['Anzahl Vorstösse'], ascending=False)
    file.to_csv('bezirke/'+bezirk+'.csv', index=False)

In [94]:
for bezirk in bezirke:
    #file = list_politicians[list_politicians["Bezirk"]=="Bezirk "+bezirk].sort_values(by=['Anzahl Vorstösse'], ascending=False)
    file.to_csv('bezirke/'+bezirk+'.csv', index=False)

In [95]:
list_politicians.groupby(by=["Partei"], dropna=False).sum()

,Anzahl Vorstösse
Partei,
Die Mitte,69
EDU,4
EVP,24
FDP,49
GLP,36
Grüne,26
SP,68
SVP,60


In [1]:
#Die Politikerinnen und Politiker mit den wenigsten Vorstössen
anz_vorst = list_politicians.groupby(by=["Name"], dropna=False).sum()
anz_vorst.sort_values(by=['Anzahl Vorstösse']).tail(15)

NameError: name 'list_politicians' is not defined

In [98]:
#Zusammenarbeit zwischen allen Parteien berechnen

alle_fraktionen=["SVP/EDU","FDP","SP","Die Mitte","GLP","Grüne","EVP"]
Zusammenarbeit = pd.DataFrame({'Parteien': alle_fraktionen})

for fraktion in alle_fraktionen:
    for fraktion2 in alle_fraktionen:
        if fraktion == fraktion2:
            pass
        else:
            anzahl_gemeinsam = 0
            fraktionen=[fraktion,fraktion2]
            for ID_Vorstoss in list_vorstoesse_namen["ID Vorstoss"].unique():
                anzahl_gemeinsam+=check_fraktionen(ID_Vorstoss,fraktionen)
            print(fraktion+" und "+fraktion2+" "+str(anzahl_gemeinsam))
                
    




SVP/EDU und FDP 18
SVP/EDU und SP 15
SVP/EDU und Die Mitte 19
SVP/EDU und GLP 12
SVP/EDU und Grüne 10
SVP/EDU und EVP 7
FDP und SVP/EDU 18
FDP und SP 17
FDP und Die Mitte 19
FDP und GLP 12
FDP und Grüne 13
FDP und EVP 8
SP und SVP/EDU 15
SP und FDP 17
SP und Die Mitte 25
SP und GLP 23
SP und Grüne 22
SP und EVP 18
Die Mitte und SVP/EDU 19
Die Mitte und FDP 19
Die Mitte und SP 25
Die Mitte und GLP 21
Die Mitte und Grüne 20
Die Mitte und EVP 16
GLP und SVP/EDU 12
GLP und FDP 12
GLP und SP 23
GLP und Die Mitte 21
GLP und Grüne 19
GLP und EVP 17
Grüne und SVP/EDU 10
Grüne und FDP 13
Grüne und SP 22
Grüne und Die Mitte 20
Grüne und GLP 19
Grüne und EVP 13
EVP und SVP/EDU 7
EVP und FDP 8
EVP und SP 18
EVP und Die Mitte 16
EVP und GLP 17
EVP und Grüne 13
